# Práctica 2: Transformaciones geométricas

### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

#### Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.

#### Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.

#### Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [11]:
#Traslacion

import cv2
import numpy as np

def nothing(x):
    pass

image = cv2.imread('images/cats.png')
size = (image.shape[1], image.shape[0])

cv2.namedWindow("Traslacion")

# traslación
cv2.createTrackbar('tx', 'Traslacion', 0, 200, nothing)
cv2.createTrackbar('ty', 'Traslacion', 0, 200, nothing)

while True:
    tx = cv2.getTrackbarPos('tx', 'Traslacion')
    ty = cv2.getTrackbarPos('ty', 'Traslacion')
    
    T = np.float32([[1,0,tx],
                    [0,1,ty]])

    imageT = cv2.warpAffine(image, T, size)

    cv2.imshow("Original", image)
    cv2.imshow('Traslacion', imageT)


    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [12]:
#Rotacion

import cv2
import numpy as np

def nothing(x):
    pass

image = cv2.imread('images/cats.png')
size = (image.shape[1], image.shape[0])

cv2.namedWindow("Rotacion")


cv2.createTrackbar('a', 'Rotacion', 0, 360, nothing)
cv2.createTrackbar('tx', 'Rotacion', 0, 200, nothing)
cv2.createTrackbar('ty', 'Rotacion', 0, 200, nothing)

while True:
    ang = cv2.getTrackbarPos('a', 'Rotacion')
    tx = cv2.getTrackbarPos('tx', 'Rotacion')
    ty = cv2.getTrackbarPos('ty', 'Rotacion')
    
    a = np.deg2rad(ang)
    
    T = np.float32(
        [[np.cos(a),-np.sin(a),tx],
         [np.sin(a),np.cos(a),ty]])

    imageR = cv2.warpAffine(image, T, size)

    cv2.imshow("Original", image)
    cv2.imshow('Rotacion', imageR)


    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

image = cv2.imread('images/cats.png')
size = (image.shape[1], image.shape[0])

cv2.namedWindow("Rotacion")

cv2.createTrackbar('sx', 'Rotacion', 0, 20, nothing)
cv2.createTrackbar('sy', 'Rotacion', 0, 20, nothing)

while True:
    sx = cv2.getTrackbarPos('sx', 'Rotacion') / 10
    sy = cv2.getTrackbarPos('sy', 'Rotacion') / 10
    
    T = np.float32(
        [[sx, 0, 0],
         [0, sy, 0]])

    imageR = cv2.warpAffine(image, T, size)

    cv2.imshow("Original", image)
    cv2.imshow('Rotacion', imageR)


    if cv2.waitKey(1) & 0xFF == 27:
        break


cv2.destroyAllWindows()
cv2.waitKey(1)


-1

### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [ ]:
# Proyección de una imagen en otra mediante homografía

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar imagen original
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco

# Copias para dibujar los puntos
clone1 = img.copy()
clone2 = img2.copy()

# Listas para guardar puntos
pts_src = []
pts_dst = []

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2

    # param = 1 → primera imagen (fuente)
    # param = 2 → segunda imagen (destino)
    if event == cv.EVENT_LBUTTONDOWN:
        if param == 1 and len(pts_src) < 4:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Original', clone1)

        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Si ya hay 4 puntos en cada imagen, proyectar
        if len(pts_src) == 4 and len(pts_dst) == 4:
            proyectar_imagen()

# ------------------------------
# Función para realizar la proyección
# ------------------------------
def proyectar_imagen():
    global pts_src, pts_dst, img, img2

    # Calcular matriz de transformación de perspectiva
    M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))

    # Aplicar transformación a la imagen original
    proyectada = cv.warpPerspective(img, M, (img2.shape[1], img2.shape[0]))

    # Mostrar la proyección
    cv.imshow('Proyeccion', proyectada)
    print("Transformación completada.")
    cv.waitKey(0)
    cv.destroyAllWindows()

# ------------------------------
# Ventanas e interacción
# ------------------------------
cv.namedWindow('Original')
cv.namedWindow('Destino')

cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("Haz clic en 4 puntos en la imagen ORIGINAL (rojo).")
print("Luego haz clic en 4 puntos en la imagen DESTINO (azul).")

cv.imshow('Original', img)
cv.imshow('Destino', img2)


while True:
    k = cv.waitKey(1) & 0xFF
    if k == 27:  
        break

cv.destroyAllWindows()

# TODO: arreglar cierre robusto


Haz clic en 4 puntos en la imagen ORIGINAL (rojo).
Luego haz clic en 4 puntos en la imagen DESTINO (azul).
Transformación completada.


KeyboardInterrupt: 

### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

##### (Optativo – 5 puntos)
- Marcar el punto de giro con el ratón.
- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en
una imagen destino y realizar la transformación afín.
- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.
- Otras aportaciones